## Ucitavanje i Sredjivanje Nizova Podataka

In [1]:
#ucitavanje potrebnih biblioteka
import pandas as pd
import numpy as np

In [2]:
#uvoz podataka
#sezona 23/24
df_23_4 = pd.read_csv("serie-a-2023.csv")
#sezona 24/25
df_24_5 = pd.read_csv("serie-a-2024.csv")
#sezona 25/6
link_25 = pd.read_html("https://fixturedownload.com/results/serie-a-2025")
df_25_6 = link_25[0]


## Ukidanje Duplih Vrijednosti za Nazive Klubova

## Određivanje Duzine Trenutnoj Sezoni i Prikaz

In [3]:
#odrediti duzinu sezone - OBAVEZNO!!!
df_25_6 = df_25_6[df_25_6["Result"] != "-"]
df_25_6.tail(5)

,Round Number,Date,Location,Home Team,Away Team,Result
151,16,20/12/2025 19:45,Juventus Stadium,Juventus,Roma,2 - 1
152,16,21/12/2025 11:30,Unipol Domus,Cagliari,Pisa,2 - 2
153,16,21/12/2025 14:00,Mapei Stadium - Città del Tricolore,Sassuolo,Torino,0 - 1
154,16,21/12/2025 17:00,Artemio Franchi,Fiorentina,Udinese,5 - 1
155,16,21/12/2025 19:45,Luigi Ferraris,Genoa,Atalanta,0 - 1


In [4]:
#sjedinjavanje ta tri df niza podataka - po osi jedan - odnosno uzduž
df_BL = pd.concat([df_23_4, df_24_5, df_25_6], axis=0)

In [5]:
df_BL["Home Team"].unique()

array(['Empoli', 'Frosinone', 'Genoa', 'Inter', 'Roma', 'Sassuolo',
       'Lecce', 'Udinese', 'Torino', 'Bologna', 'Monza', 'Milan',
       'Hellas Verona', 'Fiorentina', 'Juventus', 'Lazio', 'Napoli',
       'Salernitana', 'Cagliari', 'Atalanta', 'Parma', 'Venezia', 'Como',
       'Cremonese', 'Pisa'], dtype=object)

In [6]:
#prikaz sjedinjenog niza podatak 
#tj. sve tri sezone u jednom nizu podataka u obliku df 
df_BL.tail()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
151,NaN,16,20/12/2025 19:45,Juventus Stadium,Juventus,Roma,2 - 1
152,NaN,16,21/12/2025 11:30,Unipol Domus,Cagliari,Pisa,2 - 2
153,NaN,16,21/12/2025 14:00,Mapei Stadium - Città del Tricolore,Sassuolo,Torino,0 - 1
154,NaN,16,21/12/2025 17:00,Artemio Franchi,Fiorentina,Udinese,5 - 1
155,NaN,16,21/12/2025 19:45,Luigi Ferraris,Genoa,Atalanta,0 - 1


## Predprocesovanje Niza Podataka

In [7]:
#Izbacivanje Nerelevantnih Osobina
df_BL = df_BL.drop(["Match Number","Round Number","Date","Location"],axis=1)
df_BL.shape

(916, 3)

In [8]:
df_BL.rename(columns={"Home Team":"H","Away Team":"A"},inplace=True)

In [9]:
df_BL.to_csv("sa.csv")
df_BL

,H,A,Result
0,Empoli,Hellas Verona,0 - 1
1,Frosinone,Napoli,1 - 3
2,Genoa,Fiorentina,1 - 4
3,Inter,Monza,2 - 0
4,Roma,Salernitana,2 - 2
...,...,...,...
151,Juventus,Roma,2 - 1
152,Cagliari,Pisa,2 - 2
153,Sassuolo,Torino,0 - 1
154,Fiorentina,Udinese,5 - 1


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

#One Hot Encoder Timova - Domacin i Gost
onehot_encoder_domacin = OneHotEncoder(handle_unknown="ignore",drop="first")  # Ensure parameters are correct
kodiran_domacin = onehot_encoder_domacin.fit_transform(df_BL[["H"]]).toarray()
df_domacin = pd.DataFrame(kodiran_domacin,columns=onehot_encoder_domacin.get_feature_names_out(["H"]))

In [11]:
df_domacin.columns

Index(['H_Bologna', 'H_Cagliari', 'H_Como', 'H_Cremonese', 'H_Empoli',
       'H_Fiorentina', 'H_Frosinone', 'H_Genoa', 'H_Hellas Verona', 'H_Inter',
       'H_Juventus', 'H_Lazio', 'H_Lecce', 'H_Milan', 'H_Monza', 'H_Napoli',
       'H_Parma', 'H_Pisa', 'H_Roma', 'H_Salernitana', 'H_Sassuolo',
       'H_Torino', 'H_Udinese', 'H_Venezia'],
      dtype='object')

In [12]:
#One Hot Encoder Timova - Gosti
onehot_encoder_gost = OneHotEncoder(handle_unknown="ignore",drop="first")  # Ensure parameters are correct
kodiran_gost = onehot_encoder_gost.fit_transform(df_BL[["A"]]).toarray()
df_gost = pd.DataFrame(kodiran_gost,columns=onehot_encoder_gost.get_feature_names_out(["A"]))

In [13]:
#Sjedinjavanje tih osobina domacin i gost
df_jedan = pd.concat([df_domacin,df_gost],axis=1)
df_jedan

,H_Bologna,H_Cagliari,H_Como,H_Cremonese,H_Empoli,H_Fiorentina,H_Frosinone,H_Genoa,H_Hellas Verona,H_Inter,...,A_Monza,A_Napoli,A_Parma,A_Pisa,A_Roma,A_Salernitana,A_Sassuolo,A_Torino,A_Udinese,A_Venezia
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
912,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
913,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
914,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Transformisanje Rezultata u Varijablu - Y Varijable

#### Stvaranje Vrijednosti za Dvije Kolone iz Jedne Osobine/Kolone Rezultat

In [14]:
rez_dom = df_BL["Result"].str.slice(0,1)

In [15]:
rez_dom.head()

0    0
1    1
2    1
3    2
4    2
Name: Result, dtype: object

In [16]:
rez_gost = df_BL["Result"].str.slice(4,5)

In [17]:
rez_gost.head()

0    1
1    3
2    4
3    0
4    2
Name: Result, dtype: object

In [18]:
df_rezultatski = pd.concat([df_BL,rez_dom,rez_gost],axis=1)

In [19]:
#promjena naziva kolonama - spajanjem smo dobili kolone istih naziva
df_rezultatski.columns = ["H","A","Result","Rez1","Rez2"]
df_rezultatski.tail()

,H,A,Result,Rez1,Rez2
151,Juventus,Roma,2 - 1,2,1
152,Cagliari,Pisa,2 - 2,2,2
153,Sassuolo,Torino,0 - 1,0,1
154,Fiorentina,Udinese,5 - 1,5,1
155,Genoa,Atalanta,0 - 1,0,1


In [20]:
#spremanje df_epl varijable za obradu osobine Result u excelu
df_rezultatski.to_csv("uredjivanje_skora_vani.csv")

## Uocavanje Indeksa Sezona - za Kasnije Floatiranje

In [21]:
#df_rezultatski[0:380] #sezona 23/4 od 0:380
#df_rezultatski[380:759] #sezona 24/5 od 380:759
#df_rezultatski[760:] #sezona25/6 od 760:

## Stvaranje Kolone Skor sa Vrijednostima - Ovdje

In [22]:
# Define a function to perform the conditional logic
def calculate_value(row):
    if row['Rez1'] > row['Rez2']:
        return 1
    elif row['Rez1'] < row['Rez2']:
        return -1
    else:  # row['Column1'] == row['Column2']
        return 0

In [23]:
# Apply the function to each row in the DataFrame and assign the result to a new column
df_rezultatski['Skor'] = df_rezultatski.apply(calculate_value, axis=1)
skor = df_rezultatski.drop(['H', 'A', 'Result', 'Rez1',"Rez2"],axis=1)

### Floatiranje Vrijednosti Kolone Skor - po Sezonama

In [24]:
#sezona 23/4
skor23_4 = skor[0:380]
skor24_5 = skor[380:760]
skor25_6 = skor[760:]

In [25]:
#skor.head(3)
skor23_4["Skor"] = skor23_4["Skor"] * 0.71

C:\Users\amirb\AppData\Local\Temp\ipykernel_3728\3544292283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skor23_4["Skor"] = skor23_4["Skor"] * 0.71


In [26]:
skor24_5["Skor"] = skor24_5["Skor"] * 0.85

C:\Users\amirb\AppData\Local\Temp\ipykernel_3728\3136881136.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skor24_5["Skor"] = skor24_5["Skor"] * 0.85


## Konačna kolona Skor - Sjedinjavanje Vrijednosti Sezona

In [27]:
skor = pd.concat([skor23_4, skor24_5, skor25_6], axis=0, ignore_index=True)

In [28]:
df_rezultatski = df_rezultatski.reset_index(drop=True)
skor = skor.reset_index(drop=True)

In [29]:
#provjera rezultata
finalna_provjera = pd.concat([df_rezultatski,skor],axis=1,ignore_index=True)
finalna_provjera = finalna_provjera.drop([3,4,5],axis=1)
finalna_provjera.columns = ["H","A","Rezultat","Skor"]

In [30]:
finalna_provjera[-5:-1]

,H,A,Rezultat,Skor
911,Juventus,Roma,2 - 1,1.0
912,Cagliari,Pisa,2 - 2,0.0
913,Sassuolo,Torino,0 - 1,-1.0
914,Fiorentina,Udinese,5 - 1,1.0


# Konačni df - Podaci su Sređeni

In [31]:
skor = skor.reset_index(drop=True)
df_jedan = df_jedan.reset_index(drop=True)

In [32]:
df_BL = pd.concat([df_jedan,skor],axis=1)

In [33]:
df_BL.head(5)

,H_Bologna,H_Cagliari,H_Como,H_Cremonese,H_Empoli,H_Fiorentina,H_Frosinone,H_Genoa,H_Hellas Verona,H_Inter,...,A_Napoli,A_Parma,A_Pisa,A_Roma,A_Salernitana,A_Sassuolo,A_Torino,A_Udinese,A_Venezia,Skor
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.71
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.71
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.71
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.71
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.00


In [34]:
#spremanje kodiranog niza podataka
df_BL.to_csv("kodirani_epl.csv")

## Razdvajanje Niza Podataka na Osobine za Predvidjanje i Ciljanu Osobinu

In [35]:
# X predvidjacke, y varijabla jeste Skor Osobina
X = df_BL.drop(["Skor"],axis=1)
y = df_BL["Skor"]

In [36]:
#razdvajanje podataka u treniranju i testirajuce nizove
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=42)
X_train.head()

,H_Bologna,H_Cagliari,H_Como,H_Cremonese,H_Empoli,H_Fiorentina,H_Frosinone,H_Genoa,H_Hellas Verona,H_Inter,...,A_Monza,A_Napoli,A_Parma,A_Pisa,A_Roma,A_Salernitana,A_Sassuolo,A_Torino,A_Udinese,A_Venezia
853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
265,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
547,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
#skaliranje ovih osobina
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[-0.22587698, -0.22882899, -0.16945237, ..., -0.22882899,
        -0.22587698, -0.14944064],
       [-0.22587698, -0.22882899, -0.16945237, ..., -0.22882899,
        -0.22587698, -0.14944064],
       [-0.22587698, -0.22882899, -0.16945237, ..., -0.22882899,
        -0.22587698, -0.14944064],
       ...,
       [-0.22587698, -0.22882899, -0.16945237, ..., -0.22882899,
        -0.22587698, -0.14944064],
       [-0.22587698, -0.22882899, -0.16945237, ..., -0.22882899,
        -0.22587698,  6.69161997],
       [-0.22587698, -0.22882899, -0.16945237, ..., -0.22882899,
        -0.22587698, -0.14944064]])

In [38]:
#Spremanje tog Kodera - za Kasniju Upotrebu

with open("onehot_encoder_domacin.pkl","wb") as file:
    pickle.dump(onehot_encoder_domacin,file)
    
with open("onehot_encoder_gost.pkl","wb") as file:
    pickle.dump(onehot_encoder_gost,file)
    
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)
    

## Stvaranje Modela i Dobijanje Predikcije

In [39]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
#sledeca biblioteka se odnosi na stvaranje neurona iz kutije layers i alatom dense
from tensorflow.keras.layers import Dense

In [40]:
#konstrukcija modela
model=Sequential([
    Dense(16,activation="relu",input_shape=(X_train.shape[1],)), #HL1 connected to I/p - Skriveni Sloj 1 povezan sa I/p
    Dense(1,activation="tanh")
])

In [41]:
#Oblikovanje Modela 
model.compile(optimizer="adam",loss="mean_absolute_error",metrics=["mae"])
model.summary()

# Keras dokumentacija pojasnjava sta se dogodi sa razlicitim odabirom "loss" i drugih parametara

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                784       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 801
Trainable params: 801
Non-trainable params: 0
_________________________________________________________________


In [42]:
import datetime
#uspostavljanje Tensorboard-a, zapocinjanje treniranja
#TensorBoard sluzi samo za vizualizaciju svih logovanja koje izvrsim kod treniranja nekog modela
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

#log_dir varijabla koja osigurava da ostane zabinjezen trag svakog treniranja modela
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

#Uspostavljanje Early Stopping - sluzi za zaustavljanje treniranja ako se vrijednost gubitaka ne smanjuje
#bitni parametri su uneseni koji sluze poboljsanju funkcije ranog zaustavljanja treniranja modela...
early_stopping_callback = EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)


In [43]:
#TRENIRANJE MODELA / Tovarenje Modela
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=50,
    callbacks= [tensorflow_callback,early_stopping_callback]
)

Epoch 1/50
26/26 [==============================] - 7s 92ms/step - loss: 0.7227 - mae: 0.7227 - val_loss: 0.7083 - val_mae: 0.7083
Epoch 2/50
26/26 [==============================] - 1s 26ms/step - loss: 0.6723 - mae: 0.6723 - val_loss: 0.6807 - val_mae: 0.6807
Epoch 3/50
26/26 [==============================] - 1s 28ms/step - loss: 0.6358 - mae: 0.6358 - val_loss: 0.6534 - val_mae: 0.6534
Epoch 4/50
26/26 [==============================] - 0s 14ms/step - loss: 0.6026 - mae: 0.6026 - val_loss: 0.6372 - val_mae: 0.6372
Epoch 5/50
26/26 [==============================] - 1s 22ms/step - loss: 0.5741 - mae: 0.5741 - val_loss: 0.6186 - val_mae: 0.6186
Epoch 6/50
26/26 [==============================] - 0s 18ms/step - loss: 0.5529 - mae: 0.5529 - val_loss: 0.6014 - val_mae: 0.6014
Epoch 7/50
26/26 [==============================] - 0s 18ms/step - loss: 0.5333 - mae: 0.5333 - val_loss: 0.6032 - val_mae: 0.6032
Epoch 8/50
26/26 [==============================] - 1s 22ms/step - loss: 0.5158 - m

In [44]:
#Procjena Rada Modela preko test podataka
test_loss, test_mae = model.evaluate(X_test,y_test)

3/3 [==============================] - 0s 10ms/step - loss: 0.5914 - mae: 0.5914


In [45]:
#spremanje modela
model.save("sa_float_model.h5")

In [46]:
#moze biti da skaliranje je neophodno, onda je potrebno i obnoviti test parametara modela, te iznova
#-trenirati model i na kraju ga ponovo spremiti.

# ODREDI FORMU TIMOVA

In [47]:
#uvoz podataka
#sezona 23/24
df_23_4 = pd.read_csv("serie-a-2023.csv")
#sezona 24/25
df_24_5 = pd.read_csv("serie-a-2024.csv")
#sezona 25/6
link_25 = pd.read_html("https://fixturedownload.com/results/serie-a-2025")
df_25_6 = link_25[0]


In [48]:
#odrediti duzinu sezone - OBAVEZNO!!!
df_25_6 = df_25_6[df_25_6["Result"] != "-"]
df_25_6.tail(5)

,Round Number,Date,Location,Home Team,Away Team,Result
151,16,20/12/2025 19:45,Juventus Stadium,Juventus,Roma,2 - 1
152,16,21/12/2025 11:30,Unipol Domus,Cagliari,Pisa,2 - 2
153,16,21/12/2025 14:00,Mapei Stadium - Città del Tricolore,Sassuolo,Torino,0 - 1
154,16,21/12/2025 17:00,Artemio Franchi,Fiorentina,Udinese,5 - 1
155,16,21/12/2025 19:45,Luigi Ferraris,Genoa,Atalanta,0 - 1


In [49]:
#sjedinjavanje ta tri df niza podataka - po osi jedan - odnosno uzduž
df_BL = pd.concat([df_23_4, df_24_5, df_25_6], axis=0)
df_BL = df_BL.drop(["Match Number","Round Number","Date","Location"],axis=1)
df_BL.rename(columns={"Home Team":"H","Away Team":"A"},inplace=True)

rez_dom = df_BL["Result"].str.slice(0,1)
rez_gost = df_BL["Result"].str.slice(4,5)

df_rezultatski = pd.concat([df_BL,rez_dom,rez_gost],axis=1)
df_rezultatski.columns = ["H","A","Result","Rez1","Rez2"]

df1 = df_rezultatski

In [50]:
#definisanje funkcije...
def klub(tim):
    # Filtriramo DataFrame
    df = df1[(df1["H"] == tim) | (df1["A"] == tim)].copy()
    
    uslovi = [
    (df['H'] == tim) & (df['Rez1'] < df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] < df["Rez2"]),
    (df['H'] == tim) & (df['Rez1'] == df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] == df["Rez2"]),
    (df['H'] == tim) & (df['Rez1'] > df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] > df["Rez2"]),]
    
    vrijednosti = [-1, 1, 0, 0, 1, -1]
    df['Forma'] = np.select(uslovi, vrijednosti, default=0)
    df = df.drop(["Rez1","Rez2"],axis=1).tail(5)
    rezultat = df["Forma"].sum() / len(df["Forma"])
    return rezultat

## Provjera Rada Te Funkcije

In [51]:
klub('Inter')

0.6

In [52]:
#izvlacenje naziva svih klubova
timovi = pd.concat([df1["H"], df1["A"]]).unique()
timovi

rezultati = {}

#petlja
for tim in timovi:
    rezultat = klub(tim)
    rezultati[tim] = rezultat

# Kreiranje DataFrame-a iz rezultata
df_forme = pd.DataFrame(list(rezultati.items()),columns = ["Klub","Rezultat"])
df_forme["Rezultat"] = df_forme["Rezultat"]

## Rezultat Forme

In [53]:
#Izracun
df_forme 

,Klub,Rezultat
0,Empoli,-0.2
1,Frosinone,0.0
2,Genoa,0.0
3,Inter,0.6
4,Roma,-0.2
5,Sassuolo,-0.2
6,Lecce,0.0
7,Udinese,-0.2
8,Torino,-0.2
9,Bologna,0.0


In [54]:
#Ponovna provjera vrijednosti...
klub("Inter")

0.6

In [55]:
df_forme.to_csv("forma_h.csv")

In [56]:
df_forme["Rezultat"] = df_forme["Rezultat"] * -1

In [57]:
df_forme


,Klub,Rezultat
0,Empoli,0.2
1,Frosinone,-0.0
2,Genoa,-0.0
3,Inter,-0.6
4,Roma,0.2
5,Sassuolo,0.2
6,Lecce,-0.0
7,Udinese,0.2
8,Torino,0.2
9,Bologna,-0.0


In [58]:
df_forme.to_csv("forma_a.csv")

Kraj Skripte